In [1]:
import time


In [2]:

def mandelbrot(c, max_iter):
    z = 0
    n = 0
    while abs(z) <= 2 and n < max_iter:
        z = z**2 + c
        n += 1
    return n

def mandelbrot_set(width, height, x_min, x_max, y_min, y_max, max_iter):
    result = []
    for y in range(height):
        row = []
        for x in range(width):
            real = x_min + (x / width) * (x_max - x_min)
            imag = y_min + (y / height) * (y_max - y_min)
            c = complex(real, imag)
            row.append(mandelbrot(c, max_iter))
        result.append(row)
    return result

width = 960
height = 960
x_min, x_max = -2, 0.6
y_min, y_max = -1.5, 1.5
max_iter = 200
start_time = time.time()
result = mandelbrot_set(width, height, x_min, x_max, y_min, y_max, max_iter)
end_time = time.time()


In [3]:
elapsed_time = end_time - start_time
print(elapsed_time)

6.441244840621948


In [4]:
import time
import numpy as np
width = 960
height = 960
MAX_ITERS = 200

min_x = -2.0
max_x = 0.6
min_y = -1.5
max_y = 1.5
def mandelbrot_kernel(c):
    z = c
    for i in range(MAX_ITERS):
        z = z * z + c
        if abs(z) > 4:
            return i
    return MAX_ITERS


def compute_mandelbrot():#Tensor[float_type]:
    # create a matrix. Each element of the matrix corresponds to a pixel
    start = time.time()

    t = np.zeros((height, width))

    dx = (max_x - min_x) / width
    dy = (max_y - min_y) / height

    y = min_y
    for row in range(height):
        x = min_x
        for col in range(width):
            t[row, col] = mandelbrot_kernel(complex(x, y))
            x += dx
        y += dy
    end = time.time()
    result =end-start
    return result

In [5]:
import numpy as np
import pandas as pd
import time


def load_mnist_data(file_path):
    data = pd.read_csv(file_path)
    Y = data.iloc[:, 0].values
    X = data.iloc[:, 1:].values
    X = X / 255.0  # Normalize pixel values
    return X, Y


In [6]:
import numpy as np

def relu(x):
    return np.maximum(0, x)

def tanh(x):
    return np.tanh(x)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

def initialize_parameters(input_size, hidden_size1, hidden_size2, output_size):
    np.random.seed(42)
    parameters = {}
    parameters['W1'] = np.random.randn(input_size, hidden_size1) * 0.01
    parameters['b1'] = np.zeros((1, hidden_size1))
    parameters['W2'] = np.random.randn(hidden_size1, hidden_size2) * 0.01
    parameters['b2'] = np.zeros((1, hidden_size2))
    parameters['W3'] = np.random.randn(hidden_size2, output_size) * 0.01
    parameters['b3'] = np.zeros((1, output_size))
    return parameters

def forward_propagation(X, parameters):
    Z1 = np.dot(X, parameters['W1']) + parameters['b1']
    A1 = relu(Z1)
    Z2 = np.dot(A1, parameters['W2']) + parameters['b2']
    A2 = tanh(Z2)
    Z3 = np.dot(A2, parameters['W3']) + parameters['b3']
    A3 = softmax(Z3)
    return {'Z1': Z1, 'A1': A1, 'Z2': Z2, 'A2': A2, 'Z3': Z3, 'A3': A3}

def compute_loss(Y, probs):
    m = Y.shape[0]
    log_probs = -np.log(probs[range(m), Y])
    loss = np.sum(log_probs) / m
    return loss

def backward_propagation(X, Y, parameters, cache):
    m = X.shape[0]
    dZ3 = cache['A3'] - np.eye(10)[Y]
    dW3 = np.dot(cache['A2'].T, dZ3) / m
    db3 = np.sum(dZ3, axis=0, keepdims=True) / m
    dZ2 = np.dot(dZ3, parameters['W3'].T) * (cache['A2'] * (1 - cache['A2']))
    dW2 = np.dot(cache['A1'].T, dZ2) / m
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m
    dZ1 = np.dot(dZ2, parameters['W2'].T) * (cache['A1'] * (1 - cache['A1']))
    dW1 = np.dot(X.T, dZ1) / m
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m
    gradients = {'dW1': dW1, 'db1': db1, 'dW2': dW2, 'db2': db2, 'dW3': dW3, 'db3': db3}
    return gradients

def update_parameters(parameters, gradients, learning_rate):
    parameters['W1'] -= learning_rate * gradients['dW1']
    parameters['b1'] -= learning_rate * gradients['db1']
    parameters['W2'] -= learning_rate * gradients['dW2']
    parameters['b2'] -= learning_rate * gradients['db2']
    parameters['W3'] -= learning_rate * gradients['dW3']
    parameters['b3'] -= learning_rate * gradients['db3']
    return parameters

def train_neural_network_np(X, Y, input_size, hidden_size1, hidden_size2, output_size, learning_rate, epochs):
    parameters = initialize_parameters(input_size, hidden_size1, hidden_size2, output_size)
    
    for epoch in range(epochs):
        # Forward Propagation
        cache = forward_propagation(X, parameters)
        probs = cache['A3']

        # Compute Loss
        loss = compute_loss(Y, probs)

        # Backward Propagation
        gradients = backward_propagation(X, Y, parameters, cache)

        # Update Parameters
        parameters = update_parameters(parameters, gradients, learning_rate)

        # Print Loss every 100 epochs
        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {loss}")

    return parameters

"""# Dummy data for demonstration
input_size = 784
hidden_size1 = 150
hidden_size2 = 80
output_size = 10
learning_rate = 1e-4
epochs = 100"""
"""
# Replace X and Y with your actual data
X = np.random.randn(1000, input_size)
Y = np.random.randint(0, 10, size=(1000,))
start_time = time.time()
trained_parameters = train_neural_network(X, Y, input_size, hidden_size1, hidden_size2, output_size, learning_rate, epochs)
end = time.time()
print(end-start_time)"""

file_path = 'C:\\Users\\PC\\Desktop\\AIframework\\MNIST_test.csv'

X, Y = load_mnist_data(file_path)

# [Your existing neural network parameters]
input_size = 784
hidden_size1 = 150
hidden_size2 = 80
output_size = 10
learning_rate = 1e-4
epochs = 100

# Training the model with the MNIST data
start_time = time.time()
trained_parameters = train_neural_network_np(X, Y, input_size, hidden_size1, hidden_size2, output_size, learning_rate, epochs)
end_time = time.time()
print(f"Training time: {end_time - start_time} seconds")

Epoch 0, Loss: 2.302563784064752
Epoch 10, Loss: 2.302563382721802
Epoch 20, Loss: 2.3025629814492627
Epoch 30, Loss: 2.302562580247123
Epoch 40, Loss: 2.3025621791153665
Epoch 50, Loss: 2.30256177805398
Epoch 60, Loss: 2.30256137706295
Epoch 70, Loss: 2.3025609761422623
Epoch 80, Loss: 2.3025605752919027
Epoch 90, Loss: 2.3025601745118585
Training time: 5.4561567306518555 seconds


In [10]:
X.shape

(10000, 784)

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import time
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size1)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size1, hidden_size2)
        self.tanh = nn.Tanh()
        self.layer3 = nn.Linear(hidden_size2, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.tanh(x)
        x = self.layer3(x)
        x = self.softmax(x)
        return x

def train_neural_network_pt(X, Y, input_size, hidden_size1, hidden_size2, output_size, learning_rate, epochs):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    Y_tensor = torch.tensor(Y, dtype=torch.long)

    model = NeuralNetwork(input_size, hidden_size1, hidden_size2, output_size)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        # Forward Propagation
        outputs = model(X_tensor)

        # Compute Loss
        loss = criterion(outputs, Y_tensor)

        # Backward Propagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print Loss every 10 epochs
        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item()}")

    return model
"""
# Dummy data for demonstration
input_size = 784
hidden_size1 = 150
hidden_size2 = 80
output_size = 10
learning_rate = 1e-4
epochs = 100

# Replace X and Y with your actual data
X = np.random.randn(1000, input_size)
Y = np.random.randint(0, 10, size=(1000,))
start_time = time.time()
trained_model = train_neural_network(X, Y, input_size, hidden_size1, hidden_size2, output_size, learning_rate, epochs)
end = time.time()
print(end - start_time)
start_time = time.time()
"""


file_path = 'C:\\Users\\PC\\Desktop\\AIframework\\MNIST_test.csv'

X, Y = load_mnist_data(file_path)

# [Your existing neural network parameters]
input_size = 784
hidden_size1 = 150
hidden_size2 = 80
output_size = 10
learning_rate = 1e-4
epochs = 100

# Training the model with the MNIST data
start_time = time.time()
trained_parameters = train_neural_network_pt(X, Y, input_size, hidden_size1, hidden_size2, output_size, learning_rate, epochs)
end_time = time.time()
print(f"Training time: {end_time - start_time} seconds")

Epoch 0, Loss: 2.302729606628418
Epoch 10, Loss: 2.3027291297912598
Epoch 20, Loss: 2.3027284145355225
Epoch 30, Loss: 2.302727699279785
Epoch 40, Loss: 2.302727222442627
Epoch 50, Loss: 2.3027267456054688
Epoch 60, Loss: 2.3027260303497314
Epoch 70, Loss: 2.302725315093994
Epoch 80, Loss: 2.3027243614196777
Epoch 90, Loss: 2.3027241230010986
Training time: 1.4479990005493164 seconds


In [8]:
import tensorflow as tf
import numpy as np
import time
def create_neural_network(input_size, hidden_size1, hidden_size2, output_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(hidden_size1, activation='relu', input_shape=(input_size,)),
        tf.keras.layers.Dense(hidden_size2, activation='tanh'),
        tf.keras.layers.Dense(output_size, activation='softmax')
    ])
    return model

def train_neural_network_tf(X, Y, input_size, hidden_size1, hidden_size2, output_size, learning_rate, epochs):
    model = create_neural_network(input_size, hidden_size1, hidden_size2, output_size)
    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X, Y, epochs=epochs, batch_size=32)

    return model
"""
# Dummy data for demonstration
input_size = 784
hidden_size1 = 150
hidden_size2 = 80
output_size = 10
learning_rate = 1e-4
epochs = 10

# Replace X and Y with your actual data
X = np.random.randn(1000, input_size)
Y = np.random.randint(0, 10, size=(1000,))
start_time = time.time()
# Train the neural network
model = train_neural_network(X, Y, input_size, hidden_size1, hidden_size2, output_size, learning_rate, epochs)
end = time.time()
print(end - start_time)
"""

file_path = 'C:\\Users\\PC\\Desktop\\AIframework\\MNIST_test.csv'

X, Y = load_mnist_data(file_path)

# [Your existing neural network parameters]
input_size = 784
hidden_size1 = 150
hidden_size2 = 80
output_size = 10
learning_rate = 1e-4
epochs = 100

# Training the model with the MNIST data
start_time = time.time()
trained_parameters = train_neural_network_tf(X, Y, input_size, hidden_size1, hidden_size2, output_size, learning_rate, epochs)
end_time = time.time()
print(f"Training time: {end_time - start_time} seconds")

C:\Users\PC\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"




Epoch 1/100


313/313 [==============================] - 1s 821us/step - loss: 2.3904 - accuracy: 0.1400
Epoch 2/100
313/313 [==============================] - 0s 760us/step - loss: 2.3468 - accuracy: 0.1502
Epoch 3/100
313/313 [==============================] - 0s 782us/step - loss: 2.3057 - accuracy: 0.1623
Epoch 4/100
313/313 [==============================] - 0s 760us/step - loss: 2.2669 - accuracy: 0.1790
Epoch 5/100
313/313 [==============================] - 0s 769us/step - loss: 2.2298 - accuracy: 0.1960
Epoch 6/100
313/313 [==============================] - 0s 753us/step - loss: 2.1944 - accuracy: 0.2148
Epoch 7/100
313/313 [==============================] - 0s 754us/step - loss: 2.1603 - accuracy: 0.2404
Epoch 8/100
313/313 [==============================] - 0s 763us/step - loss: 2.1273 - accuracy: 0.2691
Epoch 9/100
313/313 [==============================] - 0s 745us/step - loss: 2.0953 - accuracy: 0.3039
Epoch 10/100
313/313 [==============================] - 0s 753us/step

313/313 [==============================] - 0s 801us/step - loss: 0.9303 - accuracy: 0.8187
Epoch 72/100
313/313 [==============================] - 0s 782us/step - loss: 0.9214 - accuracy: 0.8201
Epoch 73/100
313/313 [==============================] - 0s 780us/step - loss: 0.9127 - accuracy: 0.8211
Epoch 74/100
313/313 [==============================] - 0s 782us/step - loss: 0.9042 - accuracy: 0.8215
Epoch 75/100
313/313 [==============================] - 0s 795us/step - loss: 0.8958 - accuracy: 0.8234
Epoch 76/100
313/313 [==============================] - 0s 795us/step - loss: 0.8877 - accuracy: 0.8244
Epoch 77/100
313/313 [==============================] - 0s 785us/step - loss: 0.8797 - accuracy: 0.8258
Epoch 78/100
313/313 [==============================] - 0s 792us/step - loss: 0.8719 - accuracy: 0.8265
Epoch 79/100
313/313 [==============================] - 0s 795us/step - loss: 0.8643 - accuracy: 0.8281
Epoch 80/100
313/313 [==============================] - 0s 785us/step - loss: